In [6]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as ec
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from datetime import date, timedelta
import csv
from bs4 import BeautifulSoup
import requests
import shutil
import os
import glob
import time
import datetime
import pandas as pd
import logging
import numpy as np

In [7]:
LOGIN_PATH = f"https://plm.wwwinc.com/login"
DOWNLOAD_PATH = "D:\\User\\nguyenta\\Wolverine World Wide\\New Asia Product Operations - Data - Data\\Data Warehouse\\Attachment"
PATH=f"https://plm.wwwinc.com/criticalPathCommercializationEdit/"
USERNAME = "********"
PASSWORD = "*******"
dt_string = datetime.datetime.now().strftime("%d.%m.%Y %H%M")
data = []

In [ ]:
all_cpid=pd.read_csv(r'D:\User\nguyenta\OneDrive - Wolverine World Wide\Desktop\Comm CP ID.csv')
all_cpid

In [9]:
try:
    for cpid in all_cpid["Comm CP ID"]:
        saving_path=f"{DOWNLOAD_PATH}\\{cpid}"
        if not os.path.exists(saving_path):
            os.makedirs(saving_path)
            print(f"new folder named {cpid} was created")
except:
    pass

In [ ]:
print(f"{datetime.datetime.now()}-[get_report] start chrome object")
# Start Chrome Driver ****************************************************************
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
options.add_experimental_option("prefs",
                                {"download.default_directory": DOWNLOAD_PATH,
                                 "download.prompt_for_download": False,
                                 "download.directory_upgrade": True,
                                 "profile.default_content_settings.popups": 0,
                                 "safebrowsing.enabled": True
                                }
                                )
# options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(), options=options)
driver.implicitly_wait(300)
driver.maximize_window()
driver.get(LOGIN_PATH)
time.sleep(5)
# AUTOMATION ************************************************************************
print(f"{datetime.datetime.now()}-[get_report] login")
driver.find_element(By.NAME, "username").click()
driver.find_element(By.NAME, "username").send_keys(USERNAME)
driver.find_element(By.NAME, "password").click()
driver.find_element(By.NAME, "password").send_keys(PASSWORD)
time.sleep(3)
driver.find_element(By.XPATH,"/html/body/div/div/div/div/div/div[3]/div[3]/div/div[5]/button").click()
time.sleep(4)
data=[]
error_tab=[]
for cpid in all_cpid['Comm CP ID']:
    URL= f"{PATH}{cpid}"
    driver.get(URL)
    time.sleep(1)
    try:
        r = requests.get(URL)
        assert r.status_code==200,f"URL for {cpid} not load sucessfully"
        print(f"{datetime.datetime.now()}-[get_report] {cpid}")
        soup = BeautifulSoup(driver.page_source)
        table = soup.find('table', attrs={'class':'table table-striped table-bordered rowhover table-hove'})
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
    except AssertionError as error_msg:
        error_tab.append(cpid)
        print(f"{error_msg}")
#     print(soup)
    try:
        for row in rows:
            content={}
            content["File"] = row.find_all('td')[1].text.strip()
            content["Category"] = row.find_all('td')[2].text.strip()
            content["Size"] = row.find_all('td')[3].text.strip()
#                 Add_Factory = row.find_all('td')[4].text.strip()
#                 Colorways = row.find_all('td')[6].text.strip()
            content["Created"] = row.find_all('td')[7].text.strip()
            content["By"] = row.find_all('td')[8].text.strip()
            content["CP_ID"]=cpid
            content["load_link"]=row.find('a').get('href')
#             print( content["File"],content["Category"],content["Size"],content["Created"],content["By"],content["CP_ID"],content["load_link"])
            data.append(content)
    except:
        pass
df = pd.DataFrame(data)
df
df.to_excel(f"output_CommCP {dt_string}.xlsx",index=False)
print('Mission Completed!')
data_new=pd.read_excel(f"output_CommCP {dt_string}.xlsx")
data_new

In [ ]:
print(f"{datetime.datetime.now()}-[get_report] start run download attachment")
# Start Chrome Driver ****************************************************************
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs",
                                {"download.default_directory": DOWNLOAD_PATH,
                                 "download.prompt_for_download": False,
                                 "download.directory_upgrade": True,
                                 "profile.default_content_settings.popups": 0,
                                 "safebrowsing.enabled": True
                                }
                                )
# options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(), options=options)
driver.implicitly_wait(300)
driver.maximize_window()
data_new=pd.read_excel(f"output_CommCP {dt_string}.xlsx")
for link in data_new["load_link"].unique():
    driver.get(link)
    time.sleep(5)
# AUTOMATION ************************************************************************
    print(f"{datetime.datetime.now()}-[get_file] link {link}")


In [ ]:
print(f"{datetime.datetime.now()}-[get_report] start COPY download attachment under Comm CP ID Folder")
# Start Chrome Driver ****************************************************************
try:
    for cpid,filename in zip(data_new["CP_ID"],data_new["File"]):
        dest_path=f"{DOWNLOAD_PATH}\\{cpid}"
        file_path=f"{DOWNLOAD_PATH}\\{filename}"
        shutil.copy(file_path, dest_path)
        time.sleep(2) 
        os.remove(file_path)
        time.sleep(2)
except:
    pass